In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### 読み込み

In [ ]:
df_all = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2020/train.csv")

In [ ]:
df_all.head()

In [ ]:
df_all.columns

## データ確認

In [ ]:
df_all[['GameId', 'PlayId', 'Team', 'X', 'Y', 'S', 'A',
        'Dis', 'Orientation', 'Dir', 'NflId', 'DisplayName', 'JerseyNumber']].head()

In [ ]:
df_all[['Season', 'YardLine', 'Quarter', 'GameClock', 
        'PossessionTeam', 'Down', 'Distance', 'FieldPosition']].head()

In [ ]:
df_all[['HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'NflIdRusher', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel']].head()

In [ ]:
df_all[['PlayDirection', 'TimeHandoff','TimeSnap', 'Yards']].head()

In [ ]:
df_all[['PlayerHeight', 'PlayerWeight', 'PlayerBirthDate',
       'PlayerCollegeName', 'Position', 'HomeTeamAbbr', 'VisitorTeamAbbr']].head()

In [ ]:
df_all[['Week', 'Stadium', 'Location', 'StadiumType', 'Turf', 'GameWeather',
       'Temperature', 'Humidity', 'WindSpeed', 'WindDirection']].head()

In [ ]:
# 選手一人毎の情報がある列
# 他の列は同一PlayId内では同じ
personal_columms = ['X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir', 
                    'NflId', 'DisplayName', 'JerseyNumber', 
                   'PlayerHeight', 'PlayerWeight', 'PlayerBirthDate','PlayerCollegeName', 'Position']

In [ ]:
df_all.shape

In [ ]:
pd.set_option("display.max_columns", 80)

In [ ]:
df_all.describe()

# 前処理

In [ ]:
def preprocess(df):
#    df=pd.get_dummies(df,columns=['Team','PlayDirection','OffenseFormation','Position'])
    df=pd.get_dummies(df,columns=['OffenseFormation'])
    df=df[(df['StadiumType']!='Cloudy') & (df['StadiumType']!='Bowl')]
#    df=df.drop('FieldPosition', axis=1)
    
    # SadiumType:屋内・屋外の２分類に
    outdoor=['Outdoor', 'Outdoors','Open','Indoor, Open Roof','Outdoor Retr Roof-Open', 'Oudoor', 'Ourdoor','Retr. Roof-Open','Outdor',
       'Retr. Roof - Open', 'Domed, Open', 'Domed, open', 'Outside','Heinz Field']
    indoor=['Indoors', 'RetractableRoof', 'Indoor','Retr. Roof-Closed','Dome', 'Domed, closed','Indoor, Roof Closed', 'Retr. Roof Closed','Closed Dome','Dome, closed','Domed']
    df['stadiumtype']=(df['StadiumType'].isin(outdoor)*1)
    
    # 天候：雨か否か
    rain=['Light Rain', 'Showers','Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.','Rain', 'Heavy lake effect snow','Snow', 'Cloudy, Rain','Rain shower','Rainy']
    df['weather']=(~df['GameWeather'].isin(rain)*1)
    df.loc[ df['StadiumType'].isin(indoor), "WindSpeed" ] == 0
    
    # 身長、時計：floatに
    df['PlayerHeight']= df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['gameclock']=[ pd.Timedelta(val).total_seconds() for val in df['GameClock']]
    return df

In [ ]:
df_all = preprocess(df_all)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# ボールを持っている選手の行のみを選択
df_play = df_all[df_all["NflId"]==df_all["NflIdRusher"]]

In [ ]:
df_play["Position"].value_counts()

In [ ]:
import seaborn as sns

In [ ]:
df_play["Yards"].hist(range=(-20,30),bins=50)

## ポジションの分類

In [ ]:
df_all["Position"].unique()

In [ ]:
# オフェンスチーム
# QB
# RB（FB、HB、TB）
# WR（SE、FL、SB、WB）
# TE
# OL（C、G、T、E）
# ディフェンスチーム
# DL（DT（NT）、DE）
# LB（ILB（MLB）、OLB（LOLB、ROLB、SLB、WLB））
# DB（CB、S（SS、FS））
# スペシャルチーム
# K、P、LS、H、KR/PR

offence_position = ['WR', 'TE', 'T', 'QB', 'RB', 'G', 'C', 'FB', 'HB',  'OT', 'OG', ]
#OL_position = ['T', 'G', 'C', 'OT', 'OG']
#RB_position = ['RB', 'FB', 'HB']

defence_position = ['SS', 'DE', 'ILB', 'FS', 'CB', 'DT', 'OLB', 'NT', 'MLB', 'LB', 'S', 'DL', 'DB', 'SAF']
#DL_position = ['DL', 'DT', 'DE']
#LB_position = ['LB', 'ILB', ]

In [ ]:
df_all["offence"] = 0
df_all.loc[df_all["Position"].isin(offence_position), "offence"] = 1

## ポジション毎の人数

In [ ]:
position_count=df_all.groupby(["PlayId", "Position"]).count()
position_count

In [ ]:
df_position = position_count["GameId"].unstack().fillna(0).astype(int)
df_position

In [ ]:
df_play.loc[:, "OffensePersonnel"] = df_play["OffensePersonnel"].apply(
    lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

In [ ]:
df_play.loc[:, "DefensePersonnel"] = df_play["DefensePersonnel"].apply(
    lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

In [ ]:
# ポジション毎の人数
for position in ["DL", "LB", "DB"]:
    df_play.loc[:, position] = [ d[position] for d in df_play["DefensePersonnel"]]
for position in ["RB", "TE", "WR"]:
    df_play.loc[:, position] = [ d[position] for d in df_play["OffensePersonnel"]]    

## 選手・チームの平均情報追加

In [ ]:
# 選手の所属チーム名
df_play.loc[:, "PlayerTeamAbbr"] = df_play["HomeTeamAbbr"]
df_play.loc[ df_play["Team"]=="away", "PlayerTeamAbbr"] = df_play["VisitorTeamAbbr"]

In [ ]:
# 各チームの獲得ヤード数のSeriesのdict
team_yards_dic = dict(list(df_play[df_play["Team"]=="home"].groupby("HomeTeamAbbr")["Yards"]))

In [ ]:
# チーム毎の平均獲得ヤード（home/away別）
# 2017年度分のみ使用　
team_yards_df = df_play[(df_play["Season"]==2017) & (df_play["Team"]=="home")].groupby("HomeTeamAbbr").mean()[["Yards"]]
team_yards_df.columns = ["team_yard_av_home"]
team_yards_df.loc[:, "team_yard_av_away"] = df_play[(df_play["Season"]==2017) & (df_play["Team"]=="away")].groupby("VisitorTeamAbbr").mean()["Yards"]
team_yards_df = team_yards_df.reset_index()

In [ ]:
team_yards_df.head()

In [ ]:
df_play = df_play.merge(team_yards_df, left_on="PlayerTeamAbbr", right_on="HomeTeamAbbr", how="left")

In [ ]:
# 選手毎の平均獲得ヤード
# 2017年分のみ使用
personal_yards_2017 = df_play[df_play["Season"]==2017][["NflId", "Yards"]].groupby("NflId").agg(["mean", "std", "max", "min", "count"])["Yards"]
personal_yards_2017.dropna(inplace=True)


In [ ]:
personal_yards_2017.columns = ['rusher_yards_mean', 'rusher_yards_std', 'rusher_yards_max', 'rusher_yards_min', 'rusher_count']

In [ ]:
personal_yards_2017.head()

In [ ]:
df_play = df_play.merge(personal_yards_2017, on="NflId", how="left")

In [ ]:
df_play.loc[:, "over20"] = (df_play["Yards"]>20)*1
df_play.loc[:, "touchdown"] = (df_play["Yards"]+df_play["YardLine"]==100)

In [ ]:
# 攻撃チームの得点
df_play.loc[df_play["Team"]=="home", "rusherTeamScore"] = df_play["HomeScoreBeforePlay"]
df_play.loc[df_play["Team"]=="away", "rusherTeamScore"] = df_play["VisitorScoreBeforePlay"]

# 守備チームの得点
df_play.loc[df_play["Team"]=="home", "defenceTeamScore"] = df_play["VisitorScoreBeforePlay"]
df_play.loc[df_play["Team"]=="away", "defenceTeamScore"] = df_play["HomeScoreBeforePlay"]

# 得点差
df_play.loc[:, "diffScore"] = df_play["rusherTeamScore"] - df_play["defenceTeamScore"]

In [ ]:
# 攻撃,守備チーム平均 体重, 身長, S, A（PlayIdがキー）
offence_av = df_all.loc[df_all["offence"]==1, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
defence_av = df_all.loc[df_all["offence"]==0, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
offence_av.columns = ['PlayerHeight_offence', 'PlayerWeight_offence', 'S_offence', 'A_offence']
defence_av.columns = ['PlayerHeight_defence', 'PlayerWeight_defence', 'S_defence', 'A_defence']

In [ ]:
df_play = df_play.merge(offence_av, on="PlayId", how="left").merge(defence_av, on="PlayId", how="left")
# 列名は、_x がrusher, _y が攻撃チーム平均、添字なしが守備チーム平均になっている

In [ ]:
df_play.head()

In [ ]:
# タッチダウンまで何ヤード
df_play["yardsToTouchdown"] = df_play["YardLine"]
df_play.loc[ df_play["PossessionTeam"] == df_play["FieldPosition"], "yardsToTouchdown"
                    ] = 100-df_play["YardLine"]

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
df_play.corr()["over20"].sort_values()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm

In [ ]:
df_play.isnull().sum().sort_values().tail(20)

In [ ]:
#df_play.loc[ df_play['StadiumType'].isin(indoor), "WindSpeed" ] == 0

In [ ]:
df_use = df_play.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)

In [ ]:
df_use = df_use.fillna(df_use.mean())

# Yard数の分類モデルとして予測する

In [ ]:
df_use.loc[:, "Yards_class"] = df_use["Yards"]
df_use.loc[df_use["Yards"]>=20, "Yards_class"] = 20
df_use.loc[df_use["Yards"]<=-5, "Yards_class"] = -5

In [ ]:
from sklearn.ensemble import RandomForestClassifier as classifier


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_X = df_use.drop(["Yards", "Yards_class",  ## 答え
                    ## 文字
                    "Team", "DisplayName", "PossessionTeam", "GameClock", 
                    "OffensePersonnel", "DefensePersonnel", "DefendersInTheBox", "PlayDirection", 
                    "TimeHandoff", "TimeSnap", "PlayerBirthDate", "PlayerCollegeName", 
                    "Position", "HomeTeamAbbr_x", "VisitorTeamAbbr", "Week", 
                    "Stadium", "Location", "Turf", "PlayerTeamAbbr", "HomeTeamAbbr_y", 
                    # Yards から計算した値
                    "over20", "touchdown", 
                   # キー、番号
                   "GameId", "PlayId", "JerseyNumber", "NflId", "NflIdRusher", 
                   "WindSpeed" # 下記のとおり
                   ], axis=1)
df_y = df_use["Yards_class"]

In [ ]:
# train_X, test_X, train_y, test_y = train_test_split(df_X, df_y, test_size=0.3, shuffle=False)

In [ ]:
clf = classifier(n_estimators=100, max_depth=20, min_samples_split=10, random_state=777)
clf.fit(df_X, df_y)

In [ ]:
clf.classes_

In [ ]:
from kaggle.competitions import nflrush

In [ ]:
env = nflrush.make_env()

In [ ]:
#iter_test = env.iter_test()

In [ ]:
def count_position(df_play):
    df_play.loc[:, "OffensePersonnel"] = df_play["OffensePersonnel"].apply(
        lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )
    df_play.loc[:, "DefensePersonnel"] = df_play["DefensePersonnel"].apply(
        lambda x : { i.split(" ")[-1]:int(i.split(" ")[-2]) for i in x.split(",")} )

    # ポジション毎の人数
    for position in ["DL", "LB", "DB"]:
        df_play.loc[:, position] = [ d[position] for d in df_play["DefensePersonnel"]]
    for position in ["RB", "TE", "WR"]:
        df_play.loc[:, position] = [ d[position] for d in df_play["OffensePersonnel"]]

    return df_play
    

In [ ]:
def average_yards(df_play):
    # rusher選手の所属チーム名
    df_play.loc[:, "PlayerTeamAbbr"] = df_play["HomeTeamAbbr"]
    df_play.loc[ df_play["Team"]=="away", "PlayerTeamAbbr"] = df_play["VisitorTeamAbbr"]

    # チーム毎の平均獲得ヤード（home/away別）
    team_yards_df = df_play[(df_play["Team"]=="home")].groupby("HomeTeamAbbr").mean()[["Yards"]]
    team_yards_df.columns = ["team_yard_av_home"]
    team_yards_df.loc[:, "team_yard_av_away"] = df_play[(df_play["Team"]=="away")].groupby("VisitorTeamAbbr").mean()["Yards"]
    team_yards_df = team_yards_df.reset_index()
    
    # 選手の所属チーム
    df_play = df_play.merge(team_yards_df, left_on="PlayerTeamAbbr", right_on="HomeTeamAbbr", how="left")
    
    # 選手毎の平均獲得ヤード
    personal_yards = df_play[["NflId", "Yards"]].groupby("NflId").agg(["mean", "std", "max", "min", "count"])["Yards"]
    personal_yards.dropna(inplace=True)
    personal_yards.columns = ['rusher_yards_mean', 'rusher_yards_std', 'rusher_yards_max', 
                              'rusher_yards_min', 'rusher_count']
    
    df_play = df_play.merge(personal_yards, on="NflId", how="left")
    
    return df_play

In [ ]:
average_yard_columns = ["team_yard_av_home", "team_yard_av_away",
                        'rusher_yards_mean', 'rusher_yards_std', 'rusher_yards_max', 'rusher_yards_min',
                        'rusher_count']
df_average_yards_team = df_play[["PlayerTeamAbbr", "team_yard_av_home", "team_yard_av_away"]]
df_average_yards_rusher = df_play[['NflId', 'rusher_yards_mean', 'rusher_yards_std',
                                   'rusher_yards_max', 'rusher_yards_min','rusher_count']].copy()

In [ ]:
df_average_yards_team = df_average_yards_team.groupby("PlayerTeamAbbr").mean()
df_average_yards_rusher = df_average_yards_rusher.groupby("NflId").mean()

In [ ]:
def get_team_score(df_play):
    # 攻撃チームの得点
    df_play.loc[df_play["Team"]=="home", "rusherTeamScore"] = df_play["HomeScoreBeforePlay"]
    df_play.loc[df_play["Team"]=="away", "rusherTeamScore"] = df_play["VisitorScoreBeforePlay"]

    # 守備チームの得点
    df_play.loc[df_play["Team"]=="home", "defenceTeamScore"] = df_play["VisitorScoreBeforePlay"]
    df_play.loc[df_play["Team"]=="away", "defenceTeamScore"] = df_play["HomeScoreBeforePlay"]

    # 得点差
    df_play.loc[:, "diffScore"] = df_play["rusherTeamScore"] - df_play["defenceTeamScore"]
    
    return df_play

In [ ]:
def average_personal_data(df_play, df_all):
    # 攻撃,守備チーム平均 体重, 身長, S, A（PlayIdがキー）
    offence_av = df_all.loc[df_all["offence"]==1, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
    defence_av = df_all.loc[df_all["offence"]==0, ["PlayerHeight", "PlayerWeight", "S", "A", "PlayId"]].groupby("PlayId").mean()
    offence_av.columns = ['PlayerHeight_offence', 'PlayerWeight_offence', 'S_offence', 'A_offence']
    defence_av.columns = ['PlayerHeight_defence', 'PlayerWeight_defence', 'S_defence', 'A_defence']

    df_play = df_play.merge(offence_av, on="PlayId", how="left").merge(defence_av, on="PlayId", how="left")

    return df_play

In [ ]:
def add_last_yards(df_play):
    # タッチダウンまで何ヤード
    df_play["yardsToTouchdown"] = df_play["YardLine"]
    df_play.loc[ df_play["PossessionTeam"] == df_play["FieldPosition"], "yardsToTouchdown"
                        ] = 100-df_play["YardLine"]
    return df_play
    

In [ ]:
def select_columns(df):
    df_u = df.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)
    df_u = df_u.fillna(means)
    
#    df_use.loc[:, "Yards_class"] = df_use["Yards"]
#    df_use.loc[df_use["Yards"]>=20, "Yards_class"] = 20
#    df_use.loc[df_use["Yards"]<=-5, "Yards_class"] = -5
#'Yards' 'HomeTeamAbbr_x' 'HomeTeamAbbr_y' 'over20' 'touchdown'
    df_X = df_u.drop([#"Yards",  ## 答え
                        ## 文字
                        "Team", "DisplayName", "PossessionTeam", "GameClock", 
                        "OffensePersonnel", "DefensePersonnel", "DefendersInTheBox", "PlayDirection", 
                        "TimeHandoff", "TimeSnap", "PlayerBirthDate", "PlayerCollegeName", 
                        "Position", "HomeTeamAbbr", "VisitorTeamAbbr", "Week", 
                        "Stadium", "Location", "Turf", "PlayerTeamAbbr", #"HomeTeamAbbr_y", 
                        # Yards から計算した値
#                        "over20", "touchdown", 
                       # キー、番号
                       "GameId", "PlayId", "JerseyNumber", "NflId", "NflIdRusher", 
                       "WindSpeed" # 下記のとおり
                       ], axis=1)
    
    return df_X


# submission data 作成

In [ ]:
means = df_use.mean()

In [ ]:
train_df=df_X.iloc[:0,:]

In [ ]:
stopstop

In [ ]:

for (test_df, sample_prediction_df) in tqdm(env.iter_test()):
    df_a = preprocess(test_df)

    df_ = df_a.loc[df_a["NflId"]==df_a["NflIdRusher"], :]
    df_ = count_position(df_)

    df_.loc[:, "PlayerTeamAbbr"] = df_["HomeTeamAbbr"]
    df_.loc[ df_["Team"]=="away", "PlayerTeamAbbr"] = df_["VisitorTeamAbbr"]

    df_ = df_.merge(df_average_yards_team, on="PlayerTeamAbbr", how="left")
    df_.loc[:, df_average_yards_team.columns] = df_.fillna(df_average_yards_team.mean())
    df_ = df_.merge(df_average_yards_rusher, on="NflId", how="left")
    df_.loc[:, df_average_yards_rusher.columns] = df_.fillna(df_average_yards_rusher.mean())

    df_ = get_team_score(df_)

    df_a["offence"] = 0
    df_a.loc[df_a["Position"].isin(offence_position), "offence"] = 1
    df_ = average_personal_data(df_, df_a)
    df_ = add_last_yards(df_)

    df_u = df_.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)
    df_u = df_u.fillna(means)
    test_X = df_u.drop([#"Yards",  ## 答え
                        ## 文字
                        "Team", "DisplayName", "PossessionTeam", "GameClock", 
                        "OffensePersonnel", "DefensePersonnel", "DefendersInTheBox", "PlayDirection", 
                        "TimeHandoff", "TimeSnap", "PlayerBirthDate", "PlayerCollegeName", 
                        "Position", "HomeTeamAbbr", "VisitorTeamAbbr", "Week", 
                        "Stadium", "Location", "Turf", "PlayerTeamAbbr", #"HomeTeamAbbr_y", 
                        # Yards から計算した値
    #                        "over20", "touchdown", 
                       # キー、番号
                       "GameId", "PlayId", "JerseyNumber", "NflId", "NflIdRusher", 
                       "WindSpeed" # 下記のとおり
                       ], axis=1)
    test_X=pd.concat([train_df,test_X],sort=False)
    test_X = test_X.fillna(0)

    pred520 = clf.predict_proba(test_X)[0]
    pred100 = np.concatenate([ np.zeros(94), pred520.cumsum(), np.ones(79) ])
    sample_prediction_df.iloc[0] = pred100
    
    sample_prediction_df[sample_prediction_df>1.0] = 1.0

#    pred_df = pd.DataFrame(clf.predict_proba(test_X), columns=[ "Yards"+str(i) for i in clf.classes_])
#    pred_df = pred_df.cumsum(axis=1)

#    sample_prediction_df.iloc[0, 93:121] = pred_df.iloc[0, :]
#    sample_prediction_df.loc[:, :"Yards-6"] = 0
#    sample_prediction_df.loc[:, "Yards21":] = 1
#    sample_prediction_df.loc[:, "Yards-5":"Yards20"] = pred_df
    
    env.predict(sample_prediction_df)
    
    

In [ ]:
env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])

In [ ]:
owariowari

# テスト

In [ ]:
(test_df, sample_prediction_df) = next(env.iter_test())

In [ ]:
df_all = preprocess(test_df)


In [ ]:
df_ = df_all[df_all["NflId"]==df_all["NflIdRusher"]]

In [ ]:
df_ = count_position(df_)

In [ ]:
df_all["offence"] = 0
df_all.loc[df_all["Position"].isin(offence_position), "offence"] = 1

In [ ]:
df_.loc[:, "PlayerTeamAbbr"] = df_["HomeTeamAbbr"]
df_.loc[ df_play["Team"]=="away", "PlayerTeamAbbr"] = df_["VisitorTeamAbbr"]
df_ = df_.merge(df_average_yards_team, on="PlayerTeamAbbr", how="left")
df_.loc[:, df_average_yards_team.columns] = df_.fillna(df_average_yards_team.mean())
df_ = df_.merge(df_average_yards_rusher, on="NflId", how="left")
df_.loc[:, df_average_yards_rusher.columns] = df_.fillna(df_average_yards_rusher.mean())

In [ ]:
df_ = get_team_score(df_)
df_ = average_personal_data(df_, df_all)
df_ = add_last_yards(df_)

In [ ]:
df_u = df_.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)


In [ ]:
#means = df_use.mean()

In [ ]:
df_u = df_u.fillna(means)

In [ ]:
test_X = df_u.drop([#"Yards",  ## 答え
                    ## 文字
                    "Team", "DisplayName", "PossessionTeam", "GameClock", 
                    "OffensePersonnel", "DefensePersonnel", "DefendersInTheBox", "PlayDirection", 
                    "TimeHandoff", "TimeSnap", "PlayerBirthDate", "PlayerCollegeName", 
                    "Position", "HomeTeamAbbr", "VisitorTeamAbbr", "Week", 
                    "Stadium", "Location", "Turf", "PlayerTeamAbbr", #"HomeTeamAbbr_y", 
                    # Yards から計算した値
#                        "over20", "touchdown", 
                   # キー、番号
                   "GameId", "PlayId", "JerseyNumber", "NflId", "NflIdRusher", 
                   "WindSpeed" # 下記のとおり
                   ], axis=1)

In [ ]:
test_X=pd.concat([train_df,test_X],sort=False)
test_X = test_X.fillna(0)

In [ ]:
#test_X = select_columns(df_)


In [ ]:
test_X = test_X.columns.drop("offence")

In [ ]:
pred_df = pd.DataFrame(clf.predict_proba(test_X), columns=[ "Yards"+str(i) for i in clf.classes_])
pred_df = pred_df.cumsum(axis=1)

In [ ]:
df_a = preprocess(test_df)

df_ = df_a.loc[df_a["NflId"]==df_a["NflIdRusher"], :]
df_ = count_position(df_)

df_.loc[:, "PlayerTeamAbbr"] = df_["HomeTeamAbbr"]
df_.loc[ df_["Team"]=="away", "PlayerTeamAbbr"] = df_["VisitorTeamAbbr"]

df_ = df_.merge(df_average_yards_team, on="PlayerTeamAbbr", how="left")
df_.loc[:, df_average_yards_team.columns] = df_.fillna(df_average_yards_team.mean())
df_ = df_.merge(df_average_yards_rusher, on="NflId", how="left")
df_.loc[:, df_average_yards_rusher.columns] = df_.fillna(df_average_yards_rusher.mean())

df_ = get_team_score(df_)

df_a["offence"] = 0
df_a.loc[df_a["Position"].isin(offence_position), "offence"] = 1
df_ = average_personal_data(df_, df_a)
df_ = add_last_yards(df_)

df_u = df_.drop(["FieldPosition", "StadiumType", "GameWeather", "WindDirection"], axis=1)
df_u = df_u.fillna(means)
test_X = df_u.drop([#"Yards",  ## 答え
                    ## 文字
                    "Team", "DisplayName", "PossessionTeam", "GameClock", 
                    "OffensePersonnel", "DefensePersonnel", "DefendersInTheBox", "PlayDirection", 
                    "TimeHandoff", "TimeSnap", "PlayerBirthDate", "PlayerCollegeName", 
                    "Position", "HomeTeamAbbr", "VisitorTeamAbbr", "Week", 
                    "Stadium", "Location", "Turf", "PlayerTeamAbbr", #"HomeTeamAbbr_y", 
                    # Yards から計算した値
#                        "over20", "touchdown", 
                   # キー、番号
                   "GameId", "PlayId", "JerseyNumber", "NflId", "NflIdRusher", 
                   "WindSpeed" # 下記のとおり
                   ], axis=1)
test_X=pd.concat([train_df,test_X],sort=False)
test_X = test_X.fillna(0)

pred520 = clf.predict_proba(test_X)[0]
pred100 = np.concatenate([ np.zeros(94), pred520.cumsum(), np.ones(79) ])
sample_prediction_df.loc[0] = pred100
#pred_df = pd.DataFrame(clf.predict_proba(test_X), columns=[ "Yards"+str(i) for i in clf.classes_])
#pred_df = pred_df.cumsum(axis=1)

#sample_prediction_df.iloc[0, 95:121] = pred_df.iloc[0, :]
#sample_prediction_df.loc[:, :"Yards-6"] = 0
#sample_prediction_df.loc[:, "Yards21":] = 1

In [ ]:
pred520 = clf.predict_proba(test_X)[0]
pred100 = np.concatenate([ np.zeros(94), pred520.cumsum(), np.ones(79) ])
sample_prediction_df.loc[0] = pred100

In [ ]:
pred520 = clf.predict_proba(test_X)[0]
sample_prediction_df.loc[0] = np.concatenate([ np.zeros(94), pred520 , np.zeros(79) ])

In [ ]:
pred520 = clf.predict_proba(test_X)[0]
pred520

In [ ]:
np.zeros(2)

In [ ]:
np.concatenate([ np.zeros(94), pred520 , np.zeros(79) ])
#np.concatenate( [a1, a2])

In [ ]:
sample_prediction_df.loc[:, "Yards-10":]

In [ ]:
sample_prediction_df.loc[0]

In [ ]:
env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df.loc[:, "Yards-7":]

In [ ]:
pred_df